In [75]:
# To investigate how propofol sedation affects between-subject connectome similarity, participants’ fMRI time series were limited to 73 TRs per run to match the number of TRs per condition.
# All nodes missing in any scan were excluded from analysis (30 in total). 

In [76]:
# We then calculated the Spearman correlation of each participant’s functional connectivity pattern for the awake resting-state scan with that of all other participants. 
# We averaged the Fisher-z transforms of these values to get a single similarity value per participant.

In [2]:
import os
import random
import numpy as np
from scipy import stats
import pandas as pd
from fc_utils.load_ts import load_ts
from fc_utils.load_fc import load_fc
from fc_utils.get_edge_vals import get_edge_vals
from propofol.utils.subjects import subjects
from propofol.utils.nodes_to_drop import nodes_to_drop
import pingouin as pg

In [11]:
base_dir = '../data/censor_frames'


def format_results(res, n):
    r_val = (res[0]).round(5)
    p_val = (res[1]).round(5)
    n_minus_1 = n - 1

    return f't_{n_minus_1} r = {r_val}, p = {p_val} \n'

In [4]:
def get_subs_for_level(level):
    mot = pd.read_csv('../data/ss_out_review_compiled/ss_out_review_compiled_censor_frames.csv')
    rest = mot[(mot['run']==level)&(mot['task']=='rest')&(mot['fraction censored per run']<.5)]['sub'].values
    rest = list(rest)
    movie = mot[(mot['run']==level)&(mot['task']=='movie')&(mot['fraction censored per run']<.5)]['sub'].values
    movie = list(movie)
    return list(set(movie) & set(rest))

In [5]:
def get_subs_for_task(task):
    mot = pd.read_csv('../data/ss_out_review_compiled/ss_out_review_compiled_censor_frames.csv')
    awake = mot[(mot['run']==1)&(mot['task']==task)&(mot['fraction censored per run']<.5)]['sub'].values
    awake = list(awake)
    deep = mot[(mot['run']==3)&(mot['task']==task)&(mot['fraction censored per run']<.5)]['sub'].values
    deep = list(deep)
    return list(set(deep) & set(awake))

In [6]:
def get_fc_corr(fc1, fc2):
    # turn FC matrix into a flat list of vals
    vals_1 = get_edge_vals(fc1)
    vals_2 = get_edge_vals(fc2)
    
    res = pg.corr(vals_1, vals_2, method="spearman")
    corr = float(res.r)
    return corr

    
def get_sim_for_subject(sub_index, all_fc):
    sim_to_other_subs = []
    sub_i_fc = all_fc[sub_index]
    num_subs = all_fc.shape[0]
    
    for sub_j_index in range(num_subs):
        if sub_j_index != sub_index:
            sub_j_fc = all_fc[sub_j_index]
            sim = get_fc_corr(sub_i_fc, sub_j_fc)
            sim_to_other_subs.append(sim)
    return np.mean(sim_to_other_subs)
    
    
def get_all_similarity_values(subs, task, sedation):
    # first load all fc
    all_fc = []
    for sub in subs:
        path =  f'{base_dir}/{sub}_01_{task}_{sedation}_LPI_000.netcc'
        all_fc.append(load_fc(path, 
                              subset=[0, 73], 
                              nodes_to_drop=nodes_to_drop))
    all_fc = np.array(all_fc)
    
    # get pairwise FC similarity for each subject
    all_sim = []
    for sub_index in range(len(subs)):
        sim = get_sim_for_subject(sub_index, all_fc)
        all_sim.append(sim)
    return all_sim

In [82]:
def get_scan_length(sub, task, sedation_level):
    ts = load_ts(f'../data/censor_frames/{sub}_01_{task}_{sedation_level}_LPI_000.netts')
    return ts.shape[1]

# effect of sedation on btwn-subject connectome similarity

## running paired t-tests comparing similarity values for awake vs deep

### during rest

In [83]:
useable = get_subs_for_task('rest')
awake_vals = get_all_similarity_values(useable, 'rest', '01')
deep_vals = get_all_similarity_values(useable, 'rest', '03')
stats.ttest_rel(awake_vals, 
                deep_vals)

Ttest_relResult(statistic=6.733715925978131, pvalue=8.520768359250115e-05)

### during movie

In [84]:
useable = get_subs_for_task('movie')
awake_vals = get_all_similarity_values(useable, 'movie', '01')
deep_vals = get_all_similarity_values(useable, 'movie', '03')
stats.ttest_rel(awake_vals, 
                deep_vals)

Ttest_relResult(statistic=5.198137375536318, pvalue=0.000565462106314492)

# effect of task on btwn-subject connectome similarity

## running paired t-tests comparing similarity values for movie vs rest

### awake

In [85]:
subs = get_subs_for_level(1)
print('num_subs: ', len(subs))

rest_vals = get_all_similarity_values(subs, 'rest', '01')
movie_vals = get_all_similarity_values(subs, 'movie', '01')

stats.ttest_rel(rest_vals, 
                movie_vals)

num_subs:  16


Ttest_relResult(statistic=9.917061569159697, pvalue=5.5749425951213816e-08)

### mild

In [86]:
subs = get_subs_for_level(2)
print('num_subs: ', len(subs))

rest_vals = get_all_similarity_values(subs, 'rest', '02')
movie_vals = get_all_similarity_values(subs, 'movie', '02')

stats.ttest_rel(rest_vals, 
                movie_vals)

num_subs:  13


Ttest_relResult(statistic=1.1479991065969792, pvalue=0.2733402218061704)

### deep

In [87]:
subs = get_subs_for_level(3)
print('num_subs: ', len(subs))

rest_vals = get_all_similarity_values(subs, 'rest', '03')
movie_vals = get_all_similarity_values(subs, 'movie', '03')

stats.ttest_rel(rest_vals, 
                movie_vals)

num_subs:  8


Ttest_relResult(statistic=5.194078570315178, pvalue=0.0012611929067235905)

### recovery

In [88]:
subs = get_subs_for_level(4)
print('num_subs: ', len(subs))

rest_vals = get_all_similarity_values(subs, 'rest', '04')
movie_vals = get_all_similarity_values(subs, 'movie', '04')

stats.ttest_rel(rest_vals, 
                movie_vals)

num_subs:  15


Ttest_relResult(statistic=5.499638973131075, pvalue=7.828048296554831e-05)

# looking just at 6 with all sessions

In [12]:
six_subs = set(get_subs_for_level(1)) & set(get_subs_for_level(3)) & set(get_subs_for_level(2)) & set(get_subs_for_level(4)) 
awake_vals = get_all_similarity_values(six_subs, 'movie', '01')
deep_vals = get_all_similarity_values(six_subs, 'movie', '03')
stats.ttest_rel(awake_vals, 
                deep_vals)

Ttest_relResult(statistic=8.774447487929601, pvalue=0.00031885262942570633)

In [13]:
len(six_subs)

6

In [ ]:
#   visual inspection reveals greater connectome similarity during rest than narrative listening

In [22]:
def load_fc_wrapper(base_dir, sub, task, sedation):
    path =  f'{base_dir}/{sub}_01_{task}_{sedation}_LPI_000.netcc'
    return load_fc(path, 
                  subset=[0, 73], 
                  nodes_to_drop=nodes_to_drop)



# within individuals?
rest_sim_vals = []
sub_sim_rest = []
for sub in six_subs:
    sub_sim = []
    task = 'rest'
    rest1 = load_fc_wrapper(base_dir, sub, task, '01')
    rest2 = load_fc_wrapper(base_dir, sub, task, '02')
    rest3 = load_fc_wrapper(base_dir, sub, task, '03')
    rest4 = load_fc_wrapper(base_dir, sub, task, '04')
    sub_sim.append(get_fc_corr(rest1, rest2))
    sub_sim.append(get_fc_corr(rest1, rest3))
    sub_sim.append(get_fc_corr(rest1, rest4))
    sub_sim.append(get_fc_corr(rest2, rest3))
    sub_sim.append(get_fc_corr(rest2, rest4))
    sub_sim.append(get_fc_corr(rest3, rest4))
    
    avg = np.mean(sub_sim)
    sub_sim_rest.append(sub_sim)
    rest_sim_vals.append(avg)
    

In [23]:
movie_sim_vals = []
sub_sim_movie = []

for sub in six_subs:
    sub_sim = []
    task = 'movie'
    rest1 = load_fc_wrapper(base_dir, sub, task, '01')
    rest2 = load_fc_wrapper(base_dir, sub, task, '02')
    rest3 = load_fc_wrapper(base_dir, sub, task, '03')
    rest4 = load_fc_wrapper(base_dir, sub, task, '04')
    sub_sim.append(get_fc_corr(rest1, rest2))
    sub_sim.append(get_fc_corr(rest1, rest3))
    sub_sim.append(get_fc_corr(rest1, rest4))
    sub_sim.append(get_fc_corr(rest2, rest3))
    sub_sim.append(get_fc_corr(rest2, rest4))
    sub_sim.append(get_fc_corr(rest3, rest4))

    sub_sim_movie.append(sub_sim)
    avg = np.mean(sub_sim)
    movie_sim_vals.append(avg)

In [21]:
stats.ttest_rel(rest_sim_vals, 
                movie_sim_vals)

Ttest_relResult(statistic=2.922878331862509, pvalue=0.032903576852100075)

In [18]:
rest_sim_vals

[0.32492814177819757,
 0.4785813153744143,
 0.35017028550217016,
 0.4395309891301545,
 0.3706314496219117,
 0.3754456797447751]

In [19]:
movie_sim_vals

[0.32492814177819757,
 0.4785813153744143,
 0.35017028550217016,
 0.4395309891301545,
 0.3706314496219117,
 0.3754456797447751]

In [29]:
for sub in six_subs:
    task = 'rest'
    rest1 = load_fc_wrapper(base_dir, sub, task, '01')
    rest2 = load_fc_wrapper(base_dir, sub, task, '02')
    rest3 = load_fc_wrapper(base_dir, sub, task, '03')
    rest4 = load_fc_wrapper(base_dir, sub, task, '04')
    
    low_corr = get_fc_corr(rest1, rest4)
    high_corr = get_fc_corr(rest2, rest3)
    
    print(low_corr, high_corr)
   
    print(low_corr>high_corr)


0.45474594557308834 0.302741401731973
True
0.4651179837720249 0.5186203763058962
False
0.46010994929117993 0.2763475626303176
True
0.5406860873751529 0.5498814358752779
False
0.4584487444989151 0.3192862366660432
True
0.5159481749404468 0.45102737445765756
True


In [30]:
for sub in six_subs:
    task = 'movie'
    rest1 = load_fc_wrapper(base_dir, sub, task, '01')
    rest2 = load_fc_wrapper(base_dir, sub, task, '02')
    rest3 = load_fc_wrapper(base_dir, sub, task, '03')
    rest4 = load_fc_wrapper(base_dir, sub, task, '04')
    
    low_corr = get_fc_corr(rest1, rest4)
    high_corr = get_fc_corr(rest2, rest3)
    
    print(low_corr, high_corr)
   
    print(low_corr>high_corr)


0.43715890879294556 0.3020763038298372
True
0.39047492900491976 0.34073468402510243
True
0.29749427446960325 0.22638909663521717
True
0.3994824821180246 0.3822609836643371
True
0.36060142857992433 0.26171275497503976
True
0.3313009064760447 0.4301728980460402
False
